In [ ]:
import socket
import time
import os
import sys
import platform
from pathlib import Path
import datetime as datetime
import re

module_path = os.path.abspath(os.path.join('../../wpshelper'))
if module_path not in sys.path:
    sys.path.append(module_path)
from wpshelper import wps_open, wps_open_capture, wps_export_audio, wps_export_pcapng, wps_wireless_devices, wps_find_installations, wps_analyze_capture, wps_get_available_streams_audio, wps_close

module_path = os.path.abspath(os.path.join('../../lib'))
if module_path not in sys.path:
    sys.path.append(module_path)
from util import run_command, start_server


In [ ]:

# =======================START OF CONFIGURATION SECTION (USER MUST MODIFY TO MATCH THEIR SETTINGS)================
# See the manual Automation Server Protocol.pdf for more details
TCP_IP = '127.0.0.1'    # The IP address of the interface that the automation server is running on, normally local host 127.0.0.1
TCP_PORT = 22901        # The default port to talk with the automation server

MAX_TO_READ = 1000

# This will capture Bluetooth LE
# This is described in the section "IOParameters – Sodera and X500 of the Automation Server Protocol manual
# IMPORTANT: This is ignored for the X240. For configurations that include one or more X240 devices, the capture 
# technology should be configured by using the appropriate datasource prior to starting capture via automation. 
# Configuration of capture technology and initiating firmware update are not supported via the automation interface. 
#capture_technology = "capturetechnology=bredr-off|le-on|2m-on|spectrum-off|wifi-off"
capture_technology = "capturetechnology=bredr-off|le-on|2m-on|wifi-on|wpan-on|spectrum-on;interval=100"

# Only one personality_key should be uncommented and it should be the device that is connected and powered up before
# running the script. Additional personality keys to support other equipment and configurations 
# can be found under the section "Start FTS" in the Automation Server Protocol manual.
#
# Only uncomment one of the personality_key statements below:
# personality_key = "SODERA"
# personality_key = "X240"  
# personality_key = "X500"
# personality_key = "X500e"
personality_key = "VIEW"

# Setup the path to the the location of FTS.exe. Change this to your directory
DEFAULT_WPS_BASE_DIR = r"C:\Program Files (x86)\Teledyne LeCroy Wireless"

# ===================================END OF CONFIGURATION SECTION=====================================

wps_installations = wps_find_installations(base_dir=DEFAULT_WPS_BASE_DIR, show_log=False)
latest_install = (wps_installations or {}).get("latest") or {}

wps_path=latest_install.get("path", "")


# Automatically derive the following data
personality_key = personality_key.strip()  # remove any spaces

wps_executable_path = wps_path + '\\' +  'Executables\\Core'
# verify the path exists
if not Path(wps_executable_path).exists():
    print("WPS Executable path does not exist: " + wps_executable_path)
else:
    print("Using WPS Path: " + wps_executable_path)

In [ ]:
auto_server_process = start_server(wps_executable_path)
if auto_server_process is not None:
    print(f"auto_server_process started")

In [ ]:
wps_handle = wps_open(tcp_ip=TCP_IP,tcp_port=TCP_PORT,max_to_read = 1000,wps_executable_path=wps_executable_path,personality_key=personality_key,recv_retry_attempts=5,recv_retry_sleep=5,show_log=True)

In [ ]:
capture_absolute_filename=r'C:\Users\Public\Documents\share\input\x500_bredr_le_2m_wifi_wpan_20260205_165051.cfax'
capture_pcapng_export_filename_ex1=r'C:\Users\Public\Documents\share\input\x500_bredr_le_2m_wifi_wpan_20260205_165051_ex1.pcapng'
capture_pcapng_export_filename_ex2=r'C:\Users\Public\Documents\share\input\x500_bredr_le_2m_wifi_wpan_20260205_165051_ex2.pcapng'


In [ ]:
wps_open_capture(wps_handle, capture_absolute_filename)

In [ ]:
wps_wireless_devices(wps_handle, action="select", action_parameters={'type':'bluetooth','address':'all','select':'yes'}, show_log=True)
wps_analyze_capture(wps_handle,show_log=True,recv_retry_attempts=5, recv_retry_sleep=5)

In [ ]:
# Export pcapng
wps_export_pcapng(wps_handle, capture_pcapng_export_filename_ex1,'le')

In [ ]:
# Export pcapng
wps_export_pcapng(wps_handle, capture_pcapng_export_filename_ex2,'wpan')

In [ ]:
wps_close(wps_handle)

In [ ]:
from wpshelper import wps_write_log
wps_write_log(wps_handle, destination="stdout")